<a href="https://colab.research.google.com/github/ZHANGZ1YUE/RainfallForecasting_ConvEncDec/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Prediction
#Author: ZHANG Ziyue (Kyoto University)
#Date: 2011/11/4

from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Research/Simulation_Result/30mins_4ch


Mounted at /content/drive/
/content/drive/MyDrive/Research/Simulation_Result/30mins_4ch


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

In [ ]:
model = load_model('Unet_30mins_4ch_100ep_nodrop.h5')
input_dataset = np.load('paddedinput_30mins_4ch.npy')
output_dataset = np.load('paddedoutput_30mins_4ch.npy')

#It can predict 2 extra models, to compare the prediction accuracy of 3 models
model1 = load_model('Unet_30mins_4ch_100ep_nodrop_morefilter.h5')
model2 = load_model('Unet_30mins_4ch_100ep_nodrop_MSE.h5')

In [ ]:
#This function change the padded prediction back to unpad (180*120)
def pred_to_pad(pred, from_shape_w=192, to_shape_w=180, from_shape_h=128, to_shape_h=120):
    # pred shape 12,928,928
    padding_w = int( (from_shape_w - to_shape_w) / 2)
    padding_h = int( (from_shape_h - to_shape_h) / 2)
    return pred[::, padding_w:padding_w+to_shape_w, padding_h:padding_h+to_shape_h].copy()

#Predict
prediction = pred_to_pad(model.predict(input_dataset))

prediction1 = pred_to_pad(model1.predict(input_dataset))
prediction2 = pred_to_pad(model2.predict(input_dataset))

In [ ]:
frame = 528  #Define which specific frame you want to visualize
depad_output = pred_to_pad(output_dataset)

Image1 = depad_output[frame,:,:,0]
Image2 = prediction[frame,:,:,0]
Image3 = prediction1[frame,:,:,0]
Image4 = prediction2[frame,:,:,0]

fig = plt.figure(figsize=(16,6))

fig.add_subplot(141)
plt.imshow(Image1, cmap = 'jet', vmax = 5, vmin = 0)
plt.axis('off')
plt.title("Ground Truth", fontsize = 30, color = 'red')
#plt.colorbar()


fig.add_subplot(142)
plt.imshow(Image2, cmap = 'jet', vmax = 5, vmin = 0)
plt.axis('off')
plt.title("(Reference)", fontsize = 30, color = 'blue')

fig.add_subplot(143)
plt.imshow(Image3, cmap = 'jet', vmax = 5, vmin = 0)
plt.axis('off')
plt.title("More Filters", fontsize = 30, color = 'Green')

fig.add_subplot(144)
plt.imshow(Image4, cmap = 'jet', vmax = 5, vmin = 0)
plt.axis('off')
plt.title("MSE", fontsize = 30)


fig.tight_layout()


In [ ]:
#Display input
depad_input = pred_to_pad(input_dataset)

# reading images
Image1 = depad_input[frame,:,:,0]
Image2 = depad_input[frame,:,:,1]
Image3 = depad_input[frame,:,:,2]
Image4 = depad_input[frame,:,:,3]


fig = plt.figure(figsize=(4,6))
# Adds a subplot at the 1st position
fig.add_subplot(221)
plt.imshow(Image1, cmap = 'jet', vmax = 5, vmin = 0)
plt.axis('off')
plt.title("t-3")
  
# Adds a subplot at the 2nd position
fig.add_subplot(222)
plt.imshow(Image2, cmap = 'jet', vmax = 5, vmin = 0)
plt.axis('off')
plt.title("t-2")
  
# Adds a subplot at the 3rd position
fig.add_subplot(223)
plt.imshow(Image3, cmap = 'jet', vmax = 5, vmin = 0)
plt.axis('off')
plt.title("t-1")
  
# Adds a subplot at the 4th position
fig.add_subplot(224)
plt.imshow(Image4, cmap = 'jet', vmax = 5, vmin = 0)
plt.axis('off')
plt.title("t")


fig.tight_layout()

In [ ]:
#Calculating Mean Squared Error for 70 frames

from sklearn.metrics import mean_squared_error
error = 0

for i in range(557):
  if i < 487:
    error = error
  if i >= 487:
    error = error + mean_squared_error(prediction[i,:,:,0], depad_output[i,:,:,0])  #Here it gives the MSE for prediction, meaning its the reference model

print(error)



In [ ]:
#Visualize the intensity
def rain_intensity(data):
  intensity = np.zeros(557)
  for i in range(557):
    single_frame = 0
    if i < 487:
      single_frame = single_frame
    if i >= 487:
      for j in range(180):
        for k in range(120):                      #WHY DONT I JUST USE SUM????????????????????????  np.sum(data[0,:,:])
          single_frame = single_frame + data[i,j,k]
      intensity[i] = single_frame/180/120   #spatial average
  return intensity

plt.figure(figsize=(12,8))
plt.plot(rain_intensity(depad_output), label='ground truth')
plt.plot(rain_intensity(prediction), label='Basic')
plt.plot(rain_intensity(prediction1), label='MoreFilters')
plt.plot(rain_intensity(prediction2), label='MSE')
plt.legend()
plt.title("Spatial Averaged Rainfall Intensity", fontsize = 30, color = 'blue')
plt.xlim(487, 557)